<a href="https://colab.research.google.com/github/asmamest/tp_bigdata/blob/main/tp_spark_bigdata_version_correcte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Installation de PySpark sur Colab
!pip install pyspark

# Upload des fichiers
from google.colab import files
uploaded = files.upload()

# Step 1-2 : Initialisation
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Step 3 : Lecture du fichier Shakespeare
# Utiliser le nom exact du fichier uploadé
shakespeare_rdd = sc.textFile("Shakespeare_data.csv")
print("Type de l'opération textFile : Transformation")
print("sc : SparkContext, point d'entrée principal pour les fonctionnalités Spark")
print(f"Nombre de lignes : {shakespeare_rdd.count()}")

# Step 4 : Division en mots
words_rdd = shakespeare_rdd.flatMap(lambda line: line.split())

# Step 5 : Format clé-valeur
word_pairs_rdd = words_rdd.map(lambda word: (word, 1))

# Step 6 : Réduction par clé
word_counts_rdd = word_pairs_rdd.reduceByKey(lambda a, b: a + b)

# Step 7 : Sauvegarde
word_counts_rdd.coalesce(1).saveAsTextFile("shakespeare_word_counts")
print("Résultats sauvegardés dans 'shakespeare_word_counts'")

# Afficher les 10 mots les plus fréquents
print("10 mots les plus fréquents :")
top_words = word_counts_rdd.takeOrdered(10, key=lambda x: -x[1])
for word, count in top_words:
    print(f"{word}: {count}")

Saving Shakespeare_data.csv to Shakespeare_data.csv
Type de l'opération textFile : Transformation
sc : SparkContext, point d'entrée principal pour les fonctionnalités Spark
Nombre de lignes : 111397
Résultats sauvegardés dans 'shakespeare_word_counts'
10 mots les plus fréquents :
of: 29684
and: 27763
the: 24987
to: 14470
I: 13581
a: 11569
you: 11124
my: 10045
that: 9861
VI: 9461


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, min, max, when , avg

# Step 1 : Chargement des données météo depuis Excel
spark = SparkSession.builder.appName("WeatherAnalysis").getOrCreate()

# Méthode 1 : Convertir d'abord en CSV
import pandas as pd

# Lire le fichier Excel avec pandas
df_pandas = pd.read_excel("daily_weather.xlsx")

# Sauvegarder en CSV
df_pandas.to_csv("daily_weather.csv", index=False)

# Lire avec Spark
df = spark.read.csv("daily_weather.csv", header=True, inferSchema=True)

print("Fichier chargé avec succès!")

# Step 2 : Visualisation des colonnes
print("Colonnes :", df.columns)
print("\nSchéma :")
df.printSchema()
print("\n5 premières lignes :")
df.show(5)

# Step 3 : Statistiques récapitulatives
print("Statistiques descriptives :")
stats_df = df.describe()
stats_df.show()

# Step 4 : Calculs avancés
# 4.1 Création de la colonne ratio
df1 = df.withColumn("ratio",
                   when(col("rain_accumulation_9am") == 0, 0)
                   .otherwise(col("rain_duration_9am") / col("rain_accumulation_9am")))

# 4.2 Affichage de la colonne ratio
print("Colonne ratio :")
df1.select("ratio").show(10)

# 4.3 Maximum de rain_duration_9am
from pyspark.sql import functions as F
max_rain = df1.agg(F.max("rain_duration_9am")).first()[0]
print(f"Maximum rain_duration_9am : {max_rain}")

# 4.4 Moyenne de rain_accumulation_9am
mean_rain = df1.select(mean("rain_accumulation_9am")).first()[0]
print(f"Moyenne rain_accumulation_9am : {mean_rain}")

# 4.5 Min et Max de rain_duration_9am
min_max = df1.select(min("rain_duration_9am"), max("rain_duration_9am")).first()
print(f"Min rain_duration_9am : {min_max[0]}")
print(f"Max rain_duration_9am : {min_max[1]}")

# 4.6 Compter les températures > 70
count_temp = df1.filter(col("air_temp_9am") > 70).count()
print(f"Nombre de températures > 70 : {count_temp}")

# Step 5 : Suppression des valeurs manquantes
print(f"Lignes avant suppression des NA : {df.count()}")
df2 = df.na.drop(subset=['air_pressure_9am'])
print(f"Lignes après suppression des NA : {df2.count()}")

# Step 6 : Corrélation
correlation = df2.corr("rain_accumulation_9am", "rain_duration_9am")
print(f"Corrélation entre rain_accumulation_9am et rain_duration_9am : {correlation}")
print("Interprétation : Une corrélation proche de 0 indique une faible relation linéaire")

# Step 7 : Imputation des valeurs manquantes
imputeDF = df

print("Imputation des valeurs manquantes :")
for col_name in imputeDF.columns:
    # Vérifier si la colonne est numérique
    if dict(imputeDF.dtypes)[col_name] in ['int', 'double', 'float']:
        mean_value = imputeDF.agg(avg(col_name)).first()[0]
        imputeDF = imputeDF.na.fill(mean_value, [col_name])
        print(f"Colonne {col_name} : valeur moyenne = {mean_value}")

print("Comparaison des statistiques après imputation :")
imputeDF.describe().show()

# Fermeture
spark.stop()

Fichier chargé avec succès!
Colonnes : ['number', 'air_pressure_9am', 'air_temp_9am', 'avg_wind_direction_9am', 'avg_wind_speed_9am', 'max_wind_direction_9am', 'max_wind_speed_9am', 'rain_accumulation_9am', 'rain_duration_9am', 'relative_humidity_9am', 'relative_humidity_3pm']

Schéma :
root
 |-- number: integer (nullable = true)
 |-- air_pressure_9am: double (nullable = true)
 |-- air_temp_9am: double (nullable = true)
 |-- avg_wind_direction_9am: string (nullable = true)
 |-- avg_wind_speed_9am: double (nullable = true)
 |-- max_wind_direction_9am: double (nullable = true)
 |-- max_wind_speed_9am: double (nullable = true)
 |-- rain_accumulation_9am: double (nullable = true)
 |-- rain_duration_9am: double (nullable = true)
 |-- relative_humidity_9am: double (nullable = true)
 |-- relative_humidity_3pm: string (nullable = true)


5 premières lignes :
+------+-----------------+------------------+----------------------+------------------+----------------------+------------------+--------